<a href="https://colab.research.google.com/github/RayJohn-404notfound/Machine_learning_fire_identificaion/blob/main/CRNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install sox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sox: filename=sox-1.5.0-py3-none-any.whl size=40036 sha256=49e0eca719f416b09961dd1cc3db7e9fa8359605e11709b651e11a7a926b60d1
  Stored in directory: /root/.cache/pip/wheels/74/89/93/023fcdacaec4e5471e78b43992515e8500cc2505b307e2e6b7
Successfully built sox


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install soundfile

In [ ]:
!pip install -U snntorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 3.8 MB/s eta 0:00:00


In [ ]:
pip install torch-optimizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [ ]:
pip install torch torchvision torchaudio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import glob
import random
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import GroupShuffleSplit
import psutil
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo, nvmlShutdown
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from dataclasses import dataclass
from typing import Tuple, List


# ===================== 0. Configuration Class =====================

def get_optimal_num_workers():
    """自动计算最优的worker数量"""
    try:
        import multiprocessing
        cpu_count = multiprocessing.cpu_count()
        # 在Colab/Jupyter环境中，通常建议使用较少的worker
        # 避免创建过多进程导致性能下降
        if 'google.colab' in str(get_ipython()):
            return min(2, cpu_count)
        else:
            return min(4, cpu_count)
    except:
        return 2  # 默认安全值


@dataclass
class CRNNConfig:
    """CRNN模型和训练配置"""

    # 数据配置
    data_root: str = "/content/drive/MyDrive/data"
    train_ratio: float = 0.8
    sample_rate: int = 16000
    n_fft: int = 1024
    hop_length: int = 320
    n_mels: int = 64

    # 模型架构配置
    num_classes: int = 2
    cnn_channels: List[int] = None  # 将在__post_init__中设置
    kernel_size: int = 3
    padding: int = 1
    pool_size: Tuple[int, int] = (2, 2)

    # RNN配置
    hidden_size: int = 128
    rnn_layers: int = 2
    bidirectional: bool = True
    rnn_type: str = "GRU"  # "GRU" or "LSTM"

    # Dropout配置
    dropout_cnn: float = 0.35
    dropout_rnn: float = 0.55

    # 训练配置
    batch_size: int = 64
    learning_rate: float = 5e-5
    weight_decay: float = 1e-5
    max_epochs: int = 80
    patience: int = 25
    grad_clip_norm: float = 1.0

    # 优化器和调度器配置
    optimizer_type: str = "Adam"  # "Adam", "AdamW", "SGD"
    scheduler_type: str = "CosineAnnealing"  # "CosineAnnealing", "StepLR", "ReduceLROnPlateau"
    scheduler_T_max: int = 80
    scheduler_step_size: int = 20
    scheduler_gamma: float = 0.5
    scheduler_patience: int = 10
    scheduler_factor: float = 0.5

    # 数据加载配置
    num_workers: int = 0  # 将在__post_init__中自动设置
    pin_memory: bool = True

    # 设备和随机种子配置
    device: str = "auto"  # "auto", "cuda", "cpu"
    random_seed: int = 42

    # 保存和输出配置
    save_best_model: bool = True
    model_save_path: str = "best_model_crnn.pt"
    plot_results: bool = True
    verbose: bool = True

    def __post_init__(self):
        """初始化后处理"""
        if self.cnn_channels is None:
            self.cnn_channels = [16, 32]

        # 自动设置设备
        if self.device == "auto":
            self.device = "cuda" if torch.cuda.is_available() else "cpu"

        # 自动设置最优worker数量
        if self.num_workers == 0:
            self.num_workers = get_optimal_num_workers()
            if self.verbose:
                print(f"Auto-detected optimal num_workers: {self.num_workers}")

    def get_rnn_output_size(self) -> int:
        """计算RNN输出尺寸"""
        multiplier = 2 if self.bidirectional else 1
        return self.hidden_size * multiplier

    def print_config(self):
        """打印配置信息"""
        if not self.verbose:
            return

        print("=" * 60)
        print("CRNN Configuration Summary")
        print("=" * 60)
        print(f"Device: {self.device}")
        print(f"Random Seed: {self.random_seed}")
        print(f"Data Root: {self.data_root}")
        print(f"Train Ratio: {self.train_ratio}")
        print()
        print("Model Architecture:")
        print(f"  CNN Channels: {self.cnn_channels}")
        print(f"  RNN Type: {self.rnn_type}")
        print(f"  Hidden Size: {self.hidden_size}")
        print(f"  RNN Layers: {self.rnn_layers}")
        print(f"  Bidirectional: {self.bidirectional}")
        print(f"  Dropout CNN: {self.dropout_cnn}")
        print(f"  Dropout RNN: {self.dropout_rnn}")
        print()
        print("Training Configuration:")
        print(f"  Batch Size: {self.batch_size}")
        print(f"  Learning Rate: {self.learning_rate}")
        print(f"  Max Epochs: {self.max_epochs}")
        print(f"  Patience: {self.patience}")
        print(f"  Optimizer: {self.optimizer_type}")
        print(f"  Scheduler: {self.scheduler_type}")
        print("=" * 60)


# ===================== 1. Dataset and Preprocessing =====================

class FireDataset(Dataset):
    def __init__(self, file_list, labels, config: CRNNConfig):
        self.file_list = file_list
        self.labels = labels
        self.config = config

        self.mel_transform = torchaudio.transforms.MelSpectrogram(
            sample_rate=config.sample_rate,
            n_fft=config.n_fft,
            hop_length=config.hop_length,
            n_mels=config.n_mels,
            window_fn=torch.hann_window
        )
        self.amp_to_db = torchaudio.transforms.AmplitudeToDB(stype='power')

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_path = self.file_list[idx]
        label = self.labels[idx]

        waveform, sr = torchaudio.load(file_path)
        if sr != self.config.sample_rate:
            resampler = torchaudio.transforms.Resample(sr, self.config.sample_rate)
            waveform = resampler(waveform)

        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        mel_spec = self.mel_transform(waveform)
        mel_spec_db = self.amp_to_db(mel_spec)

        return mel_spec_db, torch.tensor(label, dtype=torch.long)


def extract_group(file_path):
    return os.path.basename(file_path).split("-")[0]


def load_data_secure(config: CRNNConfig):
    fire_dir = os.path.join(config.data_root, "fire")
    nofire_dir = os.path.join(config.data_root, "nofire")

    fire_files = glob.glob(os.path.join(fire_dir, "*.wav"))
    nofire_files = glob.glob(os.path.join(nofire_dir, "*.wav"))

    if config.verbose:
        print(f"Found {len(fire_files)} fire files, {len(nofire_files)} nofire files")

    file_list = fire_files + nofire_files
    labels = [1] * len(fire_files) + [0] * len(nofire_files)
    groups = [extract_group(f) for f in file_list]

    splitter = GroupShuffleSplit(n_splits=1, test_size=1 - config.train_ratio, random_state=config.random_seed)
    train_idx, test_idx = next(splitter.split(file_list, labels, groups=groups))

    train_files = [file_list[i] for i in train_idx]
    train_labels = [labels[i] for i in train_idx]
    test_files = [file_list[i] for i in test_idx]
    test_labels = [labels[i] for i in test_idx]

    train_dataset = FireDataset(train_files, train_labels, config)
    test_dataset = FireDataset(test_files, test_labels, config)

    return train_dataset, test_dataset


# ===================== 2. Improved CRNN Model Definition =====================

class CRNN(nn.Module):
    def __init__(self, config: CRNNConfig, input_shape=(1, 64, 128)):
        super(CRNN, self).__init__()
        self.config = config

        # 构建CNN层
        layers = []
        in_channels = input_shape[0]

        for out_channels in config.cnn_channels:
            layers.extend([
                nn.Conv2d(in_channels, out_channels,
                         kernel_size=config.kernel_size,
                         padding=config.padding),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(),
                nn.MaxPool2d(config.pool_size)
            ])
            in_channels = out_channels

        self.conv = nn.Sequential(*layers)
        self.dropout_cnn = nn.Dropout(config.dropout_cnn)

        # 动态计算RNN输入大小
        self.rnn_input_size = self._calculate_rnn_input_size(input_shape)

        # 构建RNN层
        if config.rnn_type == "GRU":
            self.rnn = nn.GRU(
                input_size=self.rnn_input_size,
                hidden_size=config.hidden_size,
                num_layers=config.rnn_layers,
                batch_first=True,
                bidirectional=config.bidirectional
            )
        elif config.rnn_type == "LSTM":
            self.rnn = nn.LSTM(
                input_size=self.rnn_input_size,
                hidden_size=config.hidden_size,
                num_layers=config.rnn_layers,
                batch_first=True,
                bidirectional=config.bidirectional
            )
        else:
            raise ValueError(f"Unsupported RNN type: {config.rnn_type}")

        self.dropout_rnn = nn.Dropout(config.dropout_rnn)

        # 注意力机制和分类层
        rnn_output_size = config.get_rnn_output_size()
        self.attn_layer = nn.Linear(rnn_output_size, 1)
        self.fc = nn.Linear(rnn_output_size, config.num_classes)

    def _calculate_rnn_input_size(self, input_shape):
        """动态计算RNN输入大小"""
        dummy_input = torch.zeros(1, *input_shape)
        with torch.no_grad():
            out = self.conv(dummy_input)
            out = self.dropout_cnn(out)
            _, channels, height, _ = out.shape
        return channels * height

    def forward(self, x):
        # CNN特征提取
        x = self.conv(x)
        x = self.dropout_cnn(x)

        # 重塑为RNN输入格式
        x = x.permute(0, 3, 1, 2)
        x = x.contiguous().view(x.size(0), x.size(1), -1)

        # RNN处理
        if self.config.rnn_type == "LSTM":
            rnn_out, _ = self.rnn(x)
        else:  # GRU
            rnn_out, _ = self.rnn(x)

        rnn_out = self.dropout_rnn(rnn_out)

        # 注意力机制
        attn_weights = torch.softmax(self.attn_layer(rnn_out), dim=1)
        context = torch.sum(attn_weights * rnn_out, dim=1)

        return self.fc(context)

    def get_attention_weights(self, x):
        """获取注意力权重用于可视化"""
        x = self.conv(x)
        x = self.dropout_cnn(x)
        x = x.permute(0, 3, 1, 2)
        x = x.contiguous().view(x.size(0), x.size(1), -1)

        if self.config.rnn_type == "LSTM":
            rnn_out, _ = self.rnn(x)
        else:
            rnn_out, _ = self.rnn(x)

        rnn_out = self.dropout_rnn(rnn_out)
        attn_weights = torch.softmax(self.attn_layer(rnn_out), dim=1)

        return attn_weights.squeeze(-1)  # [batch, time_steps]


# ===================== 3. Improved Training & Testing =====================

def create_optimizer(model, config: CRNNConfig):
    """根据配置创建优化器"""
    if config.optimizer_type == "Adam":
        return optim.Adam(model.parameters(),
                         lr=config.learning_rate,
                         weight_decay=config.weight_decay)
    elif config.optimizer_type == "AdamW":
        return optim.AdamW(model.parameters(),
                          lr=config.learning_rate,
                          weight_decay=config.weight_decay)
    elif config.optimizer_type == "SGD":
        return optim.SGD(model.parameters(),
                        lr=config.learning_rate,
                        weight_decay=config.weight_decay,
                        momentum=0.9)
    else:
        raise ValueError(f"Unsupported optimizer: {config.optimizer_type}")


def create_scheduler(optimizer, config: CRNNConfig):
    """根据配置创建学习率调度器"""
    if config.scheduler_type == "CosineAnnealing":
        return optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.scheduler_T_max)
    elif config.scheduler_type == "StepLR":
        return optim.lr_scheduler.StepLR(optimizer,
                                       step_size=config.scheduler_step_size,
                                       gamma=config.scheduler_gamma)
    elif config.scheduler_type == "ReduceLROnPlateau":
        return optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                   mode='max',
                                                   patience=config.scheduler_patience,
                                                   factor=config.scheduler_factor)
    else:
        raise ValueError(f"Unsupported scheduler: {config.scheduler_type}")


def train_model(model, train_loader, optimizer, loss_fn, config: CRNNConfig):
    model.train()
    total_loss = 0.0

    progress_bar = tqdm(train_loader, desc="Training", leave=False) if config.verbose else train_loader

    for data, target in progress_bar:
        data, target = data.to(config.device), target.to(config.device)
        output = model(data)
        loss = loss_fn(output, target)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=config.grad_clip_norm)
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)


def test_model(model, test_loader, config: CRNNConfig):
    model.eval()
    correct = 0
    total = 0

    progress_bar = tqdm(test_loader, desc="Testing", leave=False) if config.verbose else test_loader

    with torch.no_grad():
        for data, target in progress_bar:
            data, target = data.to(config.device), target.to(config.device)
            output = model(data)
            pred = torch.argmax(output, dim=1)
            correct += (pred == target).sum().item()
            total += target.size(0)

    return correct / total


# ===================== 4. Memory Monitor =====================

def print_memory_usage(config: CRNNConfig):
    if not config.verbose:
        return

    print("\n=== Resource Usage Summary ===")
    mem = psutil.virtual_memory()
    print(f"System RAM Used: {mem.percent:.2f}% ({mem.used / 1e9:.2f} GB / {mem.total / 1e9:.2f} GB)")

    if torch.cuda.is_available() and config.device == "cuda":
        try:
            nvmlInit()
            handle = nvmlDeviceGetHandleByIndex(0)
            info = nvmlDeviceGetMemoryInfo(handle)
            used_gb = info.used / 1e9
            total_gb = info.total / 1e9
            used_percent = (info.used / info.total) * 100
            print(f"GPU RAM Used: {used_percent:.2f}% ({used_gb:.2f} GB / {total_gb:.2f} GB)")
            nvmlShutdown()
        except:
            print("GPU memory info unavailable")
    else:
        print("GPU not available or not using CUDA.")


# ===================== 5. Main Function =====================

def main(config: CRNNConfig = None):
    if config is None:
        config = CRNNConfig()

    # 设置随机种子
    torch.manual_seed(config.random_seed)
    random.seed(config.random_seed)
    np.random.seed(config.random_seed)

    # 打印配置
    config.print_config()

    # 加载数据
    train_dataset, test_dataset = load_data_secure(config)

    # 创建数据加载器
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=12,
        pin_memory=config.pin_memory
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=config.batch_size,
        shuffle=False,
        drop_last=False,
        num_workers=12,
        pin_memory=config.pin_memory
    )

    # 获取输入形状
    sample_input, _ = train_dataset[0]
    input_shape = tuple(sample_input.shape)

    # 创建模型
    model = CRNN(config, input_shape).to(config.device)

    # 打印模型信息
    if config.verbose:
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"\nModel Parameters: {total_params:,} total, {trainable_params:,} trainable")

    # 创建优化器和调度器
    optimizer = create_optimizer(model, config)
    scheduler = create_scheduler(optimizer, config)
    loss_fn = nn.CrossEntropyLoss()

    # 训练循环
    best_acc = 0
    patience_counter = 0
    train_loss_list, train_acc_list, test_acc_list = [], [], []

    for epoch in range(1, config.max_epochs + 1):
        if config.verbose:
            print(f"\nEpoch {epoch}/{config.max_epochs}")

        epoch_start = time.time()

        # 训练
        train_loss = train_model(model, train_loader, optimizer, loss_fn, config)

        # 评估
        train_acc = test_model(model, train_loader, config)
        test_acc = test_model(model, test_loader, config)

        # 记录指标
        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)

        # 更新学习率
        if config.scheduler_type == "ReduceLROnPlateau":
            scheduler.step(test_acc)
        else:
            scheduler.step()

        # 打印结果
        if config.verbose:
            epoch_time = time.time() - epoch_start
            current_lr = optimizer.param_groups[0]['lr']
            print(f"Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")
            print(f"LR: {current_lr:.2e}, Time: {epoch_time:.2f}s")

        # 早停和保存最佳模型
        if test_acc > best_acc:
            best_acc = test_acc
            if config.save_best_model:
                torch.save(model.state_dict(), config.model_save_path)
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= config.patience:
                if config.verbose:
                    print("Early stopping triggered.")
                break

    if config.verbose:
        print(f"\nBest Test Accuracy: {best_acc:.4f}")

    # 打印资源使用
    print_memory_usage(config)

    # 绘制结果
    if config.plot_results:
        plot_results(train_loss_list, train_acc_list, test_acc_list, test_loader, model, config)

    return model, best_acc, (train_loss_list, train_acc_list, test_acc_list)


# ===================== 6. Plot Results (保持原有函数，但添加config参数) =====================

def plot_results(train_loss_list, train_acc_list, test_acc_list, test_loader, model, config: CRNNConfig):
    # 原有的绘图代码保持不变，但使用config.device
    epochs = range(1, len(train_acc_list) + 1)

    plt.figure(figsize=(8, 5))
    plt.plot(epochs, train_acc_list, label='Train Accuracy', color='royalblue', marker='o')
    plt.plot(epochs, test_acc_list, label='Test Accuracy', color='darkorange', marker='o')
    plt.title("Accuracy vs. Epoch", fontsize=14)
    plt.xlabel("Epoch", fontsize=12)
    plt.ylabel("Accuracy", fontsize=12)
    plt.ylim(0, 1)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig("train_vs_test_accuracy.png")
    plt.close()

    plt.figure(figsize=(8, 5))
    plt.plot(range(1, len(train_loss_list)+1), train_loss_list, label="Training Loss", color='red', marker='s')
    plt.title("Training Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.xlim(1, len(train_loss_list))
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig("training_loss.png")
    plt.close()

    # 混淆矩阵和ROC曲线代码保持不变，使用config.device
    all_preds, all_targets = [], []
    model.eval()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(config.device), target.to(config.device)
            pred = torch.argmax(model(data), dim=1)
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    cm = confusion_matrix(all_targets, all_preds)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=["nofire", "fire"], yticklabels=["nofire", "fire"])
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.tight_layout()
    plt.savefig("confusion_matrix_crnn.png")
    plt.close()

    # ROC和PR曲线
    model.eval()
    y_true, y_score = [], []

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(config.device), y.to(config.device)
            logits = model(x)
            probs = torch.softmax(logits, dim=1)[:, 1]
            y_true.extend(y.cpu().numpy())
            y_score.extend(probs.cpu().numpy())

    fpr, tpr, _ = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)
    precision, recall, _ = precision_recall_curve(y_true, y_score)
    ap = average_precision_score(y_true, y_score)

    plt.figure(figsize=(6, 5))
    plt.plot(fpr, tpr, label=f"ROC (AUC = {roc_auc:.3f})", color="royalblue")
    plt.plot([0, 1], [0, 1], "--", color="gray")

    ax2 = plt.gca().twinx()
    ax2.plot(recall, precision, color="tomato", label=f"PR (AP = {ap:.3f})")

    plt.xlabel("False-Positive Rate")
    plt.ylabel("True-Positive Rate")
    ax2.set_ylabel("Precision")
    plt.title("ROC and Precision-Recall Curves")
    plt.grid(True)
    plt.legend(loc="lower right")
    ax2.legend(loc="upper right")

    plt.tight_layout()
    plt.savefig("roc_pr_curves.png")
    plt.close()


# ===================== 7. 使用示例 =====================

if __name__ == "__main__":
    # 使用默认配置
    main()

    # 或者自定义配置
    # custom_config = CRNNConfig(
    #     learning_rate=1e-4,
    #     batch_size=32,
    #     hidden_size=256,
    #     max_epochs=100,
    #     rnn_type="LSTM"
    # )
    # main(custom_config)

Auto-detected optimal num_workers: 2
CRNN Configuration Summary
Device: cuda
Random Seed: 42
Data Root: /content/drive/MyDrive/data
Train Ratio: 0.8

Model Architecture:
  CNN Channels: [16, 32]
  RNN Type: GRU
  Hidden Size: 128
  RNN Layers: 2
  Bidirectional: True
  Dropout CNN: 0.35
  Dropout RNN: 0.55

Training Configuration:
  Batch Size: 64
  Learning Rate: 5e-05
  Max Epochs: 80
  Patience: 25
  Optimizer: Adam
  Scheduler: CosineAnnealing
Found 1697 fire files, 1562 nofire files

Model Parameters: 795,171 total, 795,171 trainable

Epoch 1/80


Loss: 0.5721, Train Acc: 0.8340, Test Acc: 0.8267
LR: 5.00e-05, Time: 106.92s

Epoch 2/80


Loss: 0.3958, Train Acc: 0.8562, Test Acc: 0.8528
LR: 4.99e-05, Time: 13.80s

Epoch 3/80


Loss: 0.2982, Train Acc: 0.9066, Test Acc: 0.9095
LR: 4.98e-05, Time: 13.73s

Epoch 4/80


Loss: 0.2341, Train Acc: 0.9305, Test Acc: 0.9371
LR: 4.97e-05, Time: 13.82s

Epoch 5/80


...

Loss: 0.0691, Train Acc: 0.9703, Test Acc: 0.9693
LR: 3.08e-05, Time: 13.61s

Epoch 35/80


Loss: 0.0734, Train Acc: 0.9691, Test Acc: 0.9663
LR: 2.99e-05, Time: 13.68s

Epoch 36/80


Loss: 0.0761, Train Acc: 0.9699, Test Acc: 0.9678
LR: 2.89e-05, Time: 13.59s

Epoch 37/80


Loss: 0.0725, Train Acc: 0.9691, Test Acc: 0.9709
LR: 2.79e-05, Time: 13.76s

Epoch 38/80


Loss: 0.0701, Train Acc: 0.9656, Test Acc: 0.9693
LR: 2.70e-05, Time: 13.99s
Early stopping triggered.

Best Test Accuracy: 0.9709

=== Resource Usage Summary ===
System RAM Used: 4.00% (2.61 GB / 89.63 GB)
GPU RAM Used: 3.94% (1.69 GB / 42.95 GB)
